In [92]:
%load_ext autoreload
%autoreload 2

from seqinf.methods.posterior import SNPE, ASNPE
from sbi.utils.user_input_checks import process_simulator

from ltu_ili_testing import GalaxySimulator, SBI_Fitter, calculate_muv
import numpy as np
import torch
from synthesizer.emission_models import PacmanEmission, TotalEmission, EmissionModel, IntrinsicEmission
from synthesizer.emission_models.attenuation import PowerLaw, Calzetti2000
from synthesizer.grid import Grid
from synthesizer.parametric import SFH, ZDist
from synthesizer.instruments import Instrument, FilterCollection, Filter
from unyt import Myr, erg, Hz, s

device = "cuda" 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Setup basic simulator model (copied from online learning notebook).

In [85]:
grid_dir = '/home/tharvey/work/synthesizer_grids/'
grid_name = 'bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps.hdf5'

grid = Grid(
    grid_name,
    grid_dir=grid_dir,
)

filter_codes = ["JWST/NIRCam.F090W","JWST/NIRCam.F115W","JWST/NIRCam.F150W","JWST/NIRCam.F162M",
                "JWST/NIRCam.F182M","JWST/NIRCam.F200W","JWST/NIRCam.F210M","JWST/NIRCam.F250M",
                "JWST/NIRCam.F277W","JWST/NIRCam.F300M","JWST/NIRCam.F335M","JWST/NIRCam.F356W",
                "JWST/NIRCam.F410M","JWST/NIRCam.F444W"]
filterset = FilterCollection(filter_codes)
instrument = Instrument('JWST', filters=filterset)

sfh = SFH.LogNormal
zdist = ZDist.DeltaConstant

emission_model = TotalEmission(
    grid=grid,
    fesc=0.1,
    fesc_ly_alpha=0.1,
    dust_curve=Calzetti2000(), 
    dust_emission_model=None,
)

# This tells the emission model we will have a parameter called 'tau_v' on the stellar emitter.
emitter_params = {'stellar':['tau_v']}

simulator = GalaxySimulator(
    sfh_model=sfh,
    zdist_model=zdist,
    grid=grid,
    instrument=instrument,
    emission_model=emission_model,
    emission_model_key='total',
    emitter_params=emitter_params,
    param_units = {'peak_age':Myr, 'max_age':Myr},
    normalize_method=calculate_muv,
    output_type='photo_fnu',
    out_flux_unit='ABmag',
)

inputs = ['redshift', 'log_mass', 'log10metallicity', 'tau_v', 'peak_age', 'max_age', 'tau']

def run_simulator(params, return_type='tensor'):
    if isinstance(params, torch.Tensor):
        params = params.cpu().numpy()

    if isinstance(params, dict):
        params = {i: params[i] for i in inputs}
    elif isinstance(params, (list, tuple, np.ndarray)):
        params = np.squeeze(params)
        params = {inputs[i]: params[i] for i in range(len(inputs))}


    phot = simulator(params)
    if return_type == 'tensor':
        return torch.tensor(phot, dtype=torch.float32).to(device)
    else:
        return phot

fitter = SBI_Fitter(
    name='online_test3',
    simulator=run_simulator,
    parameter_names=inputs,
    raw_photometry_names=simulator.instrument.filters.filter_codes+['norm'],    
)

prior_lims = {
    'redshift': (5.0, 10.0),
    'log_mass': (7.0, 10.0),
    'log10metallicity': (-3.0, 0.3),
    'tau_v': (0.0, 1.5),
    'peak_age': (0, 500),
    'max_age': (500, 1000),
    'tau': (0.3, 1.5)
}

In [37]:
priors = fitter.create_priors(override_prior_ranges=prior_lims)

---------------------------------------------
Prior ranges:
---------------------------------------------
redshift: 5.00 - 10.00
log_mass: 7.00 - 10.00
log10metallicity: -3.00 - 0.30
tau_v: 0.00 - 1.50
peak_age: 0.00 - 500.00
max_age: 500.00 - 1000.00
tau: 0.30 - 1.50
---------------------------------------------


In [77]:
from torch.distributions import Independent, Uniform
def create_priors(prior_lims, prior=Uniform):
    low = []
    high = []

    for key in prior_lims.keys():
        low.append(prior_lims[key][0])
        high.append(prior_lims[key][1])

    low = np.array(low)
    high = np.array(high)

    low = torch.tensor(low, dtype=torch.float32).to(device)
    high = torch.tensor(high, dtype=torch.float32).to(device)

    return prior(low, high)

pytorch_prior = create_priors(prior_lims)

pytorch_prior.sample()

tensor([ 8.8222e+00,  8.6660e+00, -1.7852e+00,  3.0141e-01,  3.7143e+01,
         8.3292e+02,  9.1348e-01], device='cuda:0')

In [71]:
pytorch_prior.batch_shape.numel()

7

In [72]:
priors.sample()

tensor([ 5.3598e+00,  7.5293e+00, -3.8925e-01,  8.1156e-01,  4.7654e+02,
         7.8137e+02,  1.2129e+00], device='cuda:0')

In [93]:
#safe_simulator = process_simulator(run_simulator, prior=simulators, is_numpy_simulator=True)

model = ASNPE(prior=priors, simulator=run_simulator, device='cuda:0', num_workers=1)

In [96]:
help(model)

Help on ASNPE in module seqinf.methods.posterior object:

class ASNPE(BayesianSNPE)
 |  ASNPE(*args, de_kwargs: dict | None = {'dropout_probability': 0.5}, **kwargs)
 |  
 |  ASNPE wrapper (NeurIPS 2412.05590)
 |  
 |  Method resolution order:
 |      ASNPE
 |      BayesianSNPE
 |      SNPE
 |      seqinf.sequential.SequentialInference
 |      abc.ABC
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  __annotations__ = {}
 |  
 |  collector_cls = <class 'seqinf.collector.ASNPECollector'>
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from BayesianSNPE:
 |  
 |  __init__(self, *args, de_kwargs: dict | None = {'dropout_probability': 0.5}, **kwargs)
 |      Set up multi-round inference objects.
 |      
 |      Constructs the base ``NeuralInference`` object (from the ``sbi``
 |      package) based on the provided ``method`` and ``density_estimator``,
 |      and

In [97]:
model.run(num_rounds=5, num_samples=int(1e2))

Rounds:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:seqinf.flow:Transform 1/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 2/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 3/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 4/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 5/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 6/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 7/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 8/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 9/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 10/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 11/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 1/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 2/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 3/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 4/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 5/11 [seed None] in forwar

 Training neural network. Epochs trained: 12

INFO:seqinf.flow:Transform 4/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 5/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 6/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 7/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 8/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 9/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 10/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 11/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 1/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 2/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 3/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 4/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 5/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 6/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 7/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 8/11 [seed None] in forwar

...

INFO:seqinf.flow:Transform 1/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 2/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 3/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 4/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 5/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 6/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 7/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 8/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 9/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 10/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 11/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 1/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 2/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 3/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 4/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 5/11 [seed None] in forwar

 Training neural network. Epochs trained: 529

INFO:seqinf.flow:Transform 1/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 2/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 3/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 4/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 5/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 6/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 7/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 8/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 9/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 10/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 11/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 1/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 2/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 3/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 4/11 [seed None] in forward cascade
INFO:seqinf.flow:Transform 5/11 [seed None] in forwar

 Neural network successfully converged after 533 epochs.

ValueError: Non-prior proposal needs to have x_o set; use `set_default_x` first